<a href="https://colab.research.google.com/github/peeyushsinghal/da/blob/main/mitigating_bias_sa_da_v13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mitigating bias in sentiment analysis using domain adaptation

In [1]:
! pip install torchtext==0.10.0 --quiet # DOWNGRADE YOUR TORCHTEXT
! pip install ekphrasis --quiet # library to pre process twitter data
! pip install emoji --upgrade --quiet #library to deal with emoji data

     |████████████████████████████████| 7.6 MB 5.2 MB/s 
     |████████████████████████████████| 831.4 MB 2.4 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
     |████████████████████████████████| 83 kB 1.5 MB/s 
     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 240 kB 5.0 MB/s 


In [2]:
## Import statements
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Dataset, Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe
# import torchtext.functional as TTF
import numpy as np
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
from torchtext.legacy.vocab import Vectors
from tqdm import tqdm
import random
import torch.optim as optim
# import json
# Importing library
import scipy.stats as stats
from statistics import mean

In [3]:
# checking device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:{}".format(DEVICE))

Running on:cpu


## Data loading

In [4]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data Configuration

In [5]:
BASE_PATH = '/content/drive/MyDrive/semeval-2018'

DATA_DIR = os.path.join(BASE_PATH,'datasets')
TARGET_DIR = os.path.join(BASE_PATH,'targetdataset')

MODEL_DIR = os.path.join(BASE_PATH,'models')
REF_DIR = os.path.join(BASE_PATH,'reference')

MAX_SIZE = 50
MAX_VOCAB_SIZE = 10000
BATCH_SIZE = 8

TARGET_BATCH_SIZE = 8

if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)
  print("The new directory is created!")



In [6]:
# data configuration

class TASK1(object):
  
    EI_reg = {
        'anger': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-anger-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-anger-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-anger-test-gold.txt')
                },
        'joy': {
                'train': os.path.join(
                    DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'),
                'dev': os.path.join(
                    DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-joy-dev.txt'),
                'gold': os.path.join(
                    DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-joy-test-gold.txt')
                },
        'fear': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-fear-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-fear-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-fear-test-gold.txt')
                },
        'sadness': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-sadness-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-sadness-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-sadness-test-gold.txt')
                }                     
        }

    V_reg = {
        'train': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-train.txt'),
        'dev': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-dev.txt'),
        'gold': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-test-gold.txt')
             }

    EEC = {
        'eec': os.path.join(
            DATA_DIR, 'task1/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv')
             }

## Source Data
Parsing Emotion and Valence regression data : `format [ID	Tweet	Affect Dimension	Intensity Score]`

In [7]:
def parse_reg(data_file, label_format='tuple'):#->  pd.DataFrame:
    """
    This is for datasets for the EI-reg and V-reg English tasks 
    Returns:
        df: dataframe with columns in the first row of file [ID-Tweet-Affect Dimension-Intensity Score]
    """
    with open(data_file, 'r') as fd:
      data = [l.strip().split('\t') for l in fd.readlines()]
    
    # print(data)
    df = pd.DataFrame (data[1:],columns=data[0])
    csv_file_name = (data_file.split("/")[-1]).split('.')[0]+".csv"
    csv_file = df.to_csv(str(csv_file_name))
    # df['domain'] = domain_source
    return csv_file_name
    # return df

Generic Source Data Parser

In [8]:
def parse_csv(task, dataset, emotion='anger'):
    if task == 'EI-reg':
        data_train = TASK1.EI_reg[emotion][dataset]
        csv_file_name = parse_reg(data_train)
        return csv_file_name
        # df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        # return df
    elif task == 'V-reg':
        data_train = TASK1.V_reg[dataset]
        # df = parse_reg(data_train)
        # print(data_train)
        csv_file_name = parse_reg(data_train)
        return csv_file_name
        # df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        # return df
    else:
        return None

In [9]:
emotions = ['anger','joy','fear','sadness']
dict_data ={'train':'train','dev':'val','gold':'test'}
dict_file_name ={}
for emotion in emotions:
  for data_info, data_usage in dict_data.items():
    file_name = str('file_EI_'+ emotion + "_" + data_usage)
    dict_file_name[file_name] = parse_csv('EI-reg', data_info, emotion)

    file_name2 = str('file_V_'+ data_usage)
    dict_file_name[file_name2] = parse_csv('V-reg', data_info)

# file_EI_reg_train = parse_csv('EI-reg','train','joy')
# file_EI_reg_val = parse_csv('EI-reg','dev','joy')
# file_EI_reg_test = parse_csv('EI-reg','gold','joy')

(dict_file_name)

{'file_EI_anger_train': 'EI-reg-En-anger-train.csv',
 'file_V_train': '2018-Valence-reg-En-train.csv',
 'file_EI_anger_val': '2018-EI-reg-En-anger-dev.csv',
 'file_V_val': '2018-Valence-reg-En-dev.csv',
 'file_EI_anger_test': '2018-EI-reg-En-anger-test-gold.csv',
 'file_V_test': '2018-Valence-reg-En-test-gold.csv',
 'file_EI_joy_train': 'EI-reg-En-joy-train.csv',
 'file_EI_joy_val': '2018-EI-reg-En-joy-dev.csv',
 'file_EI_joy_test': '2018-EI-reg-En-joy-test-gold.csv',
 'file_EI_fear_train': 'EI-reg-En-fear-train.csv',
 'file_EI_fear_val': '2018-EI-reg-En-fear-dev.csv',
 'file_EI_fear_test': '2018-EI-reg-En-fear-test-gold.csv',
 'file_EI_sadness_train': 'EI-reg-En-sadness-train.csv',
 'file_EI_sadness_val': '2018-EI-reg-En-sadness-dev.csv',
 'file_EI_sadness_test': '2018-EI-reg-En-sadness-test-gold.csv'}

## Preprocess tweets

In [10]:
# reference : https://github.com/cbaziotis/ekphrasis


text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [11]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#     print(" ".join(text_processor.pre_process_doc(s)))
# # print ([text_processor.pre_process_doc(s) for s in sentences])

In [12]:
def preprocess_tweet(tweet): 
  tweet_processed = text_processor.pre_process_doc(tweet)
  # print (tweet_processed)
  final_list =[]
  for index, tweet in enumerate(tweet_processed):
      final_list.append(emoji.demojize(tweet, language = 'en'))
  
  # print(df)
  return final_list

In [13]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#   print(preprocess_tweet(s))
#   # print(" ".join(preprocess_tweet(s)))

## TorchText Treatment

In [14]:
# # ORIGINAL
# field_tweet = Field(sequential=True, use_vocab = True, tokenize = preprocess_tweet, fix_length = MAX_SIZE, batch_first = True)
# field_intensity = Field(sequential= False, 
#                         dtype = torch.float,
#                         use_vocab = False 
#                         )

In [15]:
# fields = {
#     'Tweet':('tweet', field_tweet ), #
#     'Intensity Score': ('intensity',field_intensity) # Intensity Score is name of the dataset column, field_intensity is how we have defined the field, intensity is the name of the variable going fwd
#     }

In [16]:
# emotions = ['anger','joy','fear','sadness']
# dict_data ={'train':'train','dev':'val','gold':'test'}

In [17]:
dict_file_name.keys()

dict_keys(['file_EI_anger_train', 'file_V_train', 'file_EI_anger_val', 'file_V_val', 'file_EI_anger_test', 'file_V_test', 'file_EI_joy_train', 'file_EI_joy_val', 'file_EI_joy_test', 'file_EI_fear_train', 'file_EI_fear_val', 'file_EI_fear_test', 'file_EI_sadness_train', 'file_EI_sadness_val', 'file_EI_sadness_test'])

In [18]:
dict_fields ={}
list_name = list(set(["_".join(key.split("_")[1:-1]) for key in list(dict_file_name.keys())]))

for name in list_name:
  field_tweet = Field(sequential=True, 
                      use_vocab = True, 
                      tokenize = preprocess_tweet, 
                      fix_length = MAX_SIZE, 
                      batch_first = True)
  field_intensity = Field(sequential= False, 
                        dtype = torch.float,
                        use_vocab = False)
  fields = {
    'Tweet':('tweet', field_tweet ), #
    'Intensity Score': ('intensity',field_intensity) # Intensity Score is name of the dataset column, field_intensity is how we have defined the field, intensity is the name of the variable going fwd
    }
  
  dict_fields[name] = fields

  # dict_fields[name]= { 'field_tweet': Field(sequential=True,
  #                                        use_vocab = True,
  #                                        tokenize = preprocess_tweet,
  #                                        fix_length = MAX_SIZE,
  #                                        batch_first = True ), 
  #                           'field_intensity': Field(sequential= False,
  #                                              dtype = torch.float,
  #                                              use_vocab = False )}

dict_fields

{'EI_anger': {'Tweet': ('tweet',
   <torchtext.legacy.data.field.Field at 0x7f22a8431510>),
  'Intensity Score': ('intensity',
   <torchtext.legacy.data.field.Field at 0x7f22a84317d0>)},
 'EI_sadness': {'Tweet': ('tweet',
   <torchtext.legacy.data.field.Field at 0x7f22a8431790>),
  'Intensity Score': ('intensity',
   <torchtext.legacy.data.field.Field at 0x7f22a8431710>)},
 'EI_fear': {'Tweet': ('tweet',
   <torchtext.legacy.data.field.Field at 0x7f22a84316d0>),
  'Intensity Score': ('intensity',
   <torchtext.legacy.data.field.Field at 0x7f22a8431fd0>)},
 'EI_joy': {'Tweet': ('tweet',
   <torchtext.legacy.data.field.Field at 0x7f22a8431690>),
  'Intensity Score': ('intensity',
   <torchtext.legacy.data.field.Field at 0x7f22a8431610>)},
 'V': {'Tweet': ('tweet',
   <torchtext.legacy.data.field.Field at 0x7f22a84315d0>),
  'Intensity Score': ('intensity',
   <torchtext.legacy.data.field.Field at 0x7f22a8431590>)}}

In [19]:
dict_fields['EI_sadness']['Tweet'][1]

In [20]:
# WORKING CODE 
#dict_fields ={}
# list_name = list(set(["_".join(key.split("_")[1:-1]) for key in list(dict_file_name.keys())]))

# for name in list_name:
#   dict_fields[name]= { 'field_tweet': Field(sequential=True,
#                                          use_vocab = True,
#                                          tokenize = preprocess_tweet,
#                                          fix_length = MAX_SIZE,
#                                          batch_first = True ), 
#                             'field_intensity': Field(sequential= False,
#                                                dtype = torch.float,
#                                                use_vocab = False )}

# dict_fields

In [21]:
# dict_field_values = {}
# for name in list_name:
#   dict_field_values[name] = {
      
#   }


  # fields = {
#     'Tweet':('tweet', field_tweet ), #
#     'Intensity Score': ('intensity',field_intensity) # Intensity Score is name of the dataset column, field_intensity is how we have defined the field, intensity is the name of the variable going fwd
#     }

In [22]:
# dict_fields ={}
# for name in list(dict_dataset.keys()):
#   print(name,dict_dataset[name]["train_dataset"])
#   dict_fields[name]= { 'field_tweet': Field(sequential=True,
#                                          use_vocab = True,
#                                          tokenize = preprocess_tweet,
#                                          fix_length = MAX_SIZE,
#                                          batch_first = True ), 
#                             'field_intensity': Field(sequential= False,
#                                                dtype = torch.float,
#                                                use_vocab = False )}

In [23]:
dict_dataset ={}
for file_key, file_name in dict_file_name.items():
  # print(file_key,file_name)
  if "train" in (file_key.split("_")[-1]):
    head_name = "_".join(file_key.split("_")[0:-1])
    base_name = "_".join(file_key.split("_")[1:-1])
    # print(base_name)
    train_file = dict_file_name[head_name+"_train"]
    val_file = dict_file_name[head_name+"_val"]
    test_file =  dict_file_name[head_name+"_test"]

    train, val, test =TabularDataset.splits( path = './', 
                                            train = train_file, 
                                            validation = val_file, 
                                            test = test_file,
                                            format = 'csv', 
                                            fields = dict_fields[base_name])
    
    # print(train_file,val_file,test_file)
    
    # dict_dataset[base_name+"_train"], dict_dataset[base_name+"_val"],dict_dataset[base_name+"_test"]=TabularDataset.splits( path = './',
    #                                                                                                                        train = train_file,
    #                                                                                                                        validation = val_file,
    #                                                                                                                        test = test_file,
    #                                                                                                                        format = 'csv',
    #                                                                                                                        fields = fields)
    dict_dataset[base_name] = {"train_dataset": train, "val_dataset":val,"test_dataset":test}

In [24]:
# ## BEFORE USING DICT_FIELDS, USING GENERIC FIELDS HERE

# dict_dataset ={}
# for file_key, file_name in dict_file_name.items():
#   # print(file_key,file_name)
#   if "train" in (file_key.split("_")[-1]):
#     head_name = "_".join(file_key.split("_")[0:-1])
#     base_name = "_".join(file_key.split("_")[1:-1])
#     # print(base_name)
#     train_file = dict_file_name[head_name+"_train"]
#     val_file = dict_file_name[head_name+"_val"]
#     test_file =  dict_file_name[head_name+"_test"]

#     train, val, test =TabularDataset.splits( path = './', 
#                                             train = train_file, 
#                                             validation = val_file, 
#                                             test = test_file,
#                                             format = 'csv', 
#                                             fields = fields)
    
#     # print(train_file,val_file,test_file)
    
#     # dict_dataset[base_name+"_train"], dict_dataset[base_name+"_val"],dict_dataset[base_name+"_test"]=TabularDataset.splits( path = './',
#     #                                                                                                                        train = train_file,
#     #                                                                                                                        validation = val_file,
#     #                                                                                                                        test = test_file,
#     #                                                                                                                        format = 'csv',
#     #                                                                                                                        fields = fields)
#     dict_dataset[base_name] = {"train_dataset": train, "val_dataset":val,"test_dataset":test}

In [25]:
dict_dataset

{'EI_anger': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a842d610>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a76d2910>,
  'test_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a7286c50>},
 'V': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a69cb990>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a69b9690>,
  'test_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a842d650>},
 'EI_joy': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a58d4fd0>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a8455350>,
  'test_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a7fea9d0>},
 'EI_fear': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a70be090>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a752ab50>,
  'test_dataset': <torchtext.legacy

In [26]:
dict_dataset

{'EI_anger': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a842d610>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a76d2910>,
  'test_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a7286c50>},
 'V': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a69cb990>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a69b9690>,
  'test_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a842d650>},
 'EI_joy': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a58d4fd0>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a8455350>,
  'test_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a7fea9d0>},
 'EI_fear': {'train_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a70be090>,
  'val_dataset': <torchtext.legacy.data.dataset.TabularDataset at 0x7f22a752ab50>,
  'test_dataset': <torchtext.legacy

In [27]:
# train_data, valid_data, test_data = TabularDataset.splits( path = './', 
#                                               train = file_EI_reg_train,
#                                               validation = file_EI_reg_val, 
#                                               test = file_EI_reg_test,
#                                               format = 'csv',
#                                               fields = fields
#                                           )

In [28]:
# print(train_data[0].__dict__.keys(),"\n",valid_data[0].__dict__.values())

In [29]:
for key, value in dict_dataset.items():
  # count = 0
  for name, dataset in value.items():
    for example in dataset.examples:
      print(key, name, example.tweet, example.intensity)
      break


# for example in test_data.examples:
#   print(example.tweet, example.intensity)
#   count += 1
#   if count > 2:
#     break

EI_anger train_dataset ['<user>', '<user>', 'shut', 'up', 'hashtags', 'are', 'cool', '<hashtag>', 'offended', '</hashtag>'] 0.562
EI_anger val_dataset ["'", 'we', 'need', 'to', 'do', 'something', '.', 'something', 'must', 'be', 'done', '!', '<repeated>', "'", '\\', 'n', '\\', 'nyour', 'anxiety', 'is', 'amusing', '.', 'nothing', 'will', 'be', 'done', '.', 'despair', '.'] 0.517
EI_anger test_dataset ['<user>', 'i', 'know', 'you', 'mean', 'well', 'but', 'i', 'am', 'offended', '.', 'prick', '.'] 0.734
V train_dataset ['<user>', 'yeah', '!', '<happy>', 'playing', 'well'] 0.600
V val_dataset ['so', '<user>', 'site', 'crashes', 'everytime', 'i', 'try', 'to', 'book', '-', 'how', 'do', 'they', 'help', '?', 'tell', 'me', 'there', "'", 's', 'nothing', 'wrong', '&', 'hang', 'up', '<hashtag>', 'furious', '</hashtag>', '<hashtag>', 'helpless', '</hashtag>', '<user>'] 0.141
V test_dataset ['gm', 'and', 'have', 'a', '<hashtag>', 'tuesday', '</hashtag>', '!'] 0.589
EI_joy train_dataset ['<user>', 'quit

## Building iterator and Vocabulary

In [30]:
# #WORKING
# dict_fields ={}
# for name in list(dict_dataset.keys()):
#   print(name,dict_dataset[name]["train_dataset"])
#   dict_fields[name]= { 'field_tweet': Field(sequential=True,
#                                          use_vocab = True,
#                                          tokenize = preprocess_tweet,
#                                          fix_length = MAX_SIZE,
#                                          batch_first = True ), 
#                             'field_intensity': Field(sequential= False,
#                                                dtype = torch.float,
#                                                use_vocab = False )}
#   # dict_fields[name]['field_intensity'] = Field(sequential= False,
#   #                                              dtype = torch.float,
#   #                                              use_vocab = False )

# # field_tweet = Field(sequential=True,
# #                     use_vocab = True,
# #                     tokenize = preprocess_tweet, 
# #                     fix_length = MAX_SIZE, 
# #                     batch_first = True
# #                     )

# # field_intensity = Field(sequential= False, 
# #                         dtype = torch.float,
# #                         use_vocab = False 
# #                         )

In [31]:
for name, value in dict_fields.items():
  print(name, value['Tweet'][1])
  value['Tweet'][1].build_vocab(dict_dataset[name]['train_dataset'],
                                   max_size = MAX_VOCAB_SIZE,
                                   min_freq = 1,
                                   vectors = "glove.6B.100d",
                                   unk_init=torch.Tensor.normal_)
  value['Intensity Score'][1].build_vocab(dict_dataset[name]['train_dataset'])
  # value['field_tweet'].build_vocab(dict_dataset[name]['train_dataset'],
  #                                  max_size = MAX_VOCAB_SIZE,
  #                                  min_freq = 1,
  #                                  vectors = "glove.6B.100d",
  #                                  unk_init=torch.Tensor.normal_)
  # value['field_intensity'].build_vocab(dict_dataset[name]['train_dataset'])

EI_anger <torchtext.legacy.data.field.Field object at 0x7f22a8431510>


.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 399999/400000 [00:17<00:00, 23463.17it/s]


EI_sadness <torchtext.legacy.data.field.Field object at 0x7f22a8431790>
EI_fear <torchtext.legacy.data.field.Field object at 0x7f22a84316d0>
EI_joy <torchtext.legacy.data.field.Field object at 0x7f22a8431690>
V <torchtext.legacy.data.field.Field object at 0x7f22a84315d0>


In [32]:
# #WORKING
# for name, value in dict_fields.items():
#   value['field_tweet'].build_vocab(dict_dataset[name]['train_dataset'],
#                                    max_size = MAX_VOCAB_SIZE,
#                                    min_freq = 1,
#                                    vectors = "glove.6B.100d",
#                                    unk_init=torch.Tensor.normal_)
#   value['field_intensity'].build_vocab(dict_dataset[name]['train_dataset'])

In [33]:
## ORIGINAL
# field_tweet.build_vocab(train_data, 
#                   max_size = MAX_VOCAB_SIZE,
#                   min_freq = 1,
#                   vectors = "glove.6B.100d",
#                   unk_init=torch.Tensor.normal_)

# vocab = field_tweet.build_vocab(train_data, 
#                   max_size = MAX_VOCAB_SIZE,
#                   min_freq = 1,
#                   vectors = "glove.6B.100d",
#                   unk_init=torch.Tensor.normal_)
# field_intensity.build_vocab(train_data)

In [34]:
dict_iterator ={}
for name, value in dict_dataset.items():
  VALID_TEST_BATCH_SIZE= min(len(value['val_dataset']),len(value['test_dataset']) )
  print(name, VALID_TEST_BATCH_SIZE)
  train_iterator, val_iterator, test_iterator= BucketIterator.splits(
      (value['train_dataset'], value['val_dataset'],value['test_dataset']),
      batch_sizes= (BATCH_SIZE,VALID_TEST_BATCH_SIZE,VALID_TEST_BATCH_SIZE),
      sort_key = lambda x: len(x.tweet),
      sort_within_batch=True,
      device = DEVICE,
      shuffle= True)
  
  dict_iterator[name] = {"train_iterator": train_iterator, "val_iterator":val_iterator,"test_iterator":test_iterator}

  # dict_iterator[name]['train_iterator'], dict_iterator[name]['val_iterator'], dict_iterator[name]['test_iterator'] = BucketIterator.splits((dict_dataset[name]['train_dataset'], dict_dataset[name]['val_dataset'],dict_dataset[name]['test_dataset']), 
  #                                                     batch_sizes= (BATCH_SIZE,VALID_TEST_BATCH_SIZE,VALID_TEST_BATCH_SIZE),
  #                                                     sort_key = lambda x: len(x.tweet),
  #                                                     sort_within_batch=True,
  #                                                     device = DEVICE,
  #                                                     shuffle= True)

EI_anger 388
V 449
EI_joy 290
EI_fear 389
EI_sadness 397


In [35]:
dict_iterator.items()

dict_items([('EI_anger', {'train_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1f90>, 'val_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1f10>, 'test_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1dd0>}), ('V', {'train_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1610>, 'val_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1c90>, 'test_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1c50>}), ('EI_joy', {'train_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1b90>, 'val_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1b10>, 'test_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1ad0>}), ('EI_fear', {'train_iterator': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f22a14a1a10>, 'val_iterator': <torchtex

In [36]:
# VALID_TEST_BATCH_SIZE = min (len(valid_data),len(test_data))
# # print(VALID_TEST_BATCH_SIZE)

# train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
#                                                       batch_sizes= (BATCH_SIZE,VALID_TEST_BATCH_SIZE,VALID_TEST_BATCH_SIZE),
#                                                       sort_key = lambda x: len(x.tweet),
#                                                       sort_within_batch=True,
#                                                       device = DEVICE,
#                                                       shuffle= True)

In [37]:
for key, value in dict_iterator.items():
  for name, iterator in value.items():
    for batch in iterator:
      print(key, name, batch.tweet)
      print(batch.intensity)
      break
    break
  break
    
# count = 0a
# for batch in train_iterator:
#   print (batch.tweet)
#   print (batch.intensity)
#   count += 1
#   if count > 2:
#     break

EI_anger train_iterator tensor([[  31, 1818,  463, 1569,  117,   42,   30,  272,   31,   42,  216,   30,
           18,   24,   31,  107,  103,   41,  921,   30,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [  39, 1305,    8,    3,  637,    2,   10,  673,    8,    3,  508,    2,
           10,   11,  673,    8,    3,   59,    2,    4,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [ 413,   10,   72,   93,   31,  193,   30,   15, 1213,   26,  263,   12,
           22,  156,  623,    4,   24,    3,  273,    2,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,   

In [38]:
for name in list_name:
  print(dict_fields[name]['Tweet'][1].vocab.stoi.items())

dict_items([('<unk>', 0), ('<pad>', 1), ('</hashtag>', 2), ('<hashtag>', 3), ('.', 4), ('<user>', 5), ('i', 6), ('the', 7), ('to', 8), ('a', 9), (',', 10), ('and', 11), ("'", 12), ('you', 13), ('not', 14), ('is', 15), ('of', 16), ('it', 17), ('!', 18), ('that', 19), ('my', 20), ('in', 21), ('s', 22), ('me', 23), ('<repeated>', 24), ('<number>', 25), ('on', 26), ('for', 27), ('?', 28), ('do', 29), ('</allcaps>', 30), ('<allcaps>', 31), ('are', 32), ('have', 33), ('-', 34), ('am', 35), ('just', 36), ('but', 37), ('with', 38), ('be', 39), ('like', 40), ('so', 41), ('at', 42), ('all', 43), ('this', 44), ('\\', 45), ('your', 46), ('no', 47), ('can', 48), ('he', 49), ('will', 50), ('up', 51), ('out', 52), ('if', 53), ('people', 54), ('when', 55), ('get', 56), ('was', 57), ('what', 58), ('anger', 59), ('angry', 60), ('they', 61), ('about', 62), ('as', 63), ('n', 64), ('we', 65), ('rage', 66), ('by', 67), ('one', 68), ('an', 69), ('bitter', 70), ('from', 71), ('who', 72), ('&', 73), ('his', 74

In [39]:
# print(list(field_tweet.vocab.stoi.items()))
# print(field_tweet.vocab.vectors[field_tweet.vocab.stoi['the']])

In [40]:
# for batch_number, batch in enumerate(train_iterator):
#   print(batch_number)
#   print(batch.tweet)
#   print(batch.intensity)

## TorchText treatment of Target Data

In [46]:
# Target Data
df_target = pd.read_csv(os.path.join(TARGET_DIR,'winomt_saunders_combined.csv'))
print(df_target.head())
target_file_name = 'target_csv.csv'
target_csv_file = df_target.to_csv(str(target_file_name))

                             Tweet  Intensity Score
0    the actor finished her work .                0
1    the actor finished his work .                0
2  the actuary finished her work .                0
3  the actuary finished his work .                0
4    the agent finished her work .                0


In [47]:
dict_target_dataset = {}
for name in list_name:
  # print(dict_fields[name])
  dict_target_dataset [name] = TabularDataset( path = os.path.join(TARGET_DIR,'winomt_saunders_combined.csv'), 
                                              format = 'csv',
                                              fields = dict_fields[name] )

print(dict_target_dataset)

{'EI_anger': <torchtext.legacy.data.dataset.TabularDataset object at 0x7f22a04afdd0>, 'EI_sadness': <torchtext.legacy.data.dataset.TabularDataset object at 0x7f22a04b1290>, 'EI_fear': <torchtext.legacy.data.dataset.TabularDataset object at 0x7f22a017bbd0>, 'EI_joy': <torchtext.legacy.data.dataset.TabularDataset object at 0x7f229ff4f1d0>, 'V': <torchtext.legacy.data.dataset.TabularDataset object at 0x7f229f6c9d10>}


In [48]:
# target_data = TabularDataset( path = os.path.join(TARGET_DIR,'winomt_saunders_combined.csv'), 
#                                               format = 'csv',
#                                               fields = fields
#                                           )
# print(target_data[0].__dict__.keys())
# print(target_data[0].__dict__.values())

In [49]:
for name, dataset in dict_target_dataset.items():
  count=0
  for example in dataset:
    print(example.tweet, example.intensity)
    count += 1
    if count > 2:
      break

['the', 'actor', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'his', 'work', '.'] 0
['the', 'actuary', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'his', 'work', '.'] 0
['the', 'actuary', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'his', 'work', '.'] 0
['the', 'actuary', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'his', 'work', '.'] 0
['the', 'actuary', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'his', 'work', '.'] 0
['the', 'actuary', 'finished', 'her', 'work', '.'] 0


In [31]:
# count = 0
# for example in target_data:
#   print(example.tweet, example.intensity)
#   count += 1
#   if count > 2:
#     break

['the', 'actor', 'finished', 'her', 'work', '.'] 0
['the', 'actor', 'finished', 'his', 'work', '.'] 0
['the', 'actuary', 'finished', 'her', 'work', '.'] 0


In [50]:
dict_target_iterator = {}
for name in list_name:
  dict_target_iterator [name] = BucketIterator(dict_target_dataset[name], # given that there is only one dataset we are not using splits
                                 batch_size= TARGET_BATCH_SIZE,
                                 sort_key = lambda x: len(x.tweet),
                                 sort_within_batch=True,
                                 device = DEVICE,
                                 repeat=True,
                                 shuffle= True)

print(dict_target_iterator)

{'EI_anger': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f229ea3be50>, 'EI_sadness': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f229ea3ba10>, 'EI_fear': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f229ea3b250>, 'EI_joy': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f229ea3bf90>, 'V': <torchtext.legacy.data.iterator.BucketIterator object at 0x7f229ea3bad0>}


In [32]:

# target_iterator = BucketIterator(target_data, # given that there is only one dataset we are not using splits
#                                  batch_size= TARGET_BATCH_SIZE,
#                                  sort_key = lambda x: len(x.tweet),
#                                  sort_within_batch=True,
#                                  device = DEVICE,
#                                  repeat=True,
#                                  shuffle= True)

In [54]:
# next(iter(target_iterator))

In [52]:
for name, iterator in dict_target_iterator.items():
  count = 0
  for batch in iterator:
    print(name)
    print(batch)
    print (batch.tweet)
    print (batch.intensity)
    count += 1
    break
    if count > 2:
      break

EI_anger

[torchtext.legacy.data.batch.Batch of size 8]
	[.tweet]:[torch.LongTensor of size 8x50]
	[.intensity]:[torch.FloatTensor of size 8]
tensor([[   7,    0,  679,    7,    0,    8,  213,  118,  570,  100,   49,   57,
            0, 1106,  117,    4,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   7,    0,  679,  113,    8, 1776,   52,    9,    0,   41,   80,  255,
           39,    0, 2641,    4,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   7,    0,  679,    7,    0,    8,  408,  155,   79,   63,   49,   15,
            0,    7,  257,    4,    1,    1,    1,    1,    1,    1,    1,    1,
    

In [ ]:
# count = 0
# for batch in target_iterator:
#   print(batch)
#   print (batch.tweet)
#   print (batch.intensity)
#   count += 1
#   if count > 2:
#     break

## CNN 1d model

### Gradient Reversal layer

In [68]:
from torch.autograd import Function


class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

CNN 1 D model
Reference: A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification, Ye Zhang, Byron Wallace 2015

Difference:

use of embedding
use of sigmoid function, as we are having a regression model not a classififer as the main task

In [60]:
import torch.nn as nn
import torch.nn.functional as F

class CNN1d(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim, 
                 n_filters, 
                 filter_sizes, 
                 output_dim, 
                 dropout, 
                 pad_idx
                 ):
        super().__init__()
        
        #---------------------Feature Extractor Network----------------------#
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        # Convolutional Network
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        #---------------------Regression Network------------------------#
        # Fully-connected layer and Dropout
        self.regression = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(len(filter_sizes) * n_filters, len(filter_sizes) * n_filters // 2),
            nn.ReLU(),
            nn.Linear(len(filter_sizes) * n_filters // 2, output_dim * 10),
            nn.ReLU(),
            nn.Linear(output_dim * 10, output_dim)
            # ,
            # nn.Sigmoid()
        )
        # self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim * 10)
        # self.fc2 = nn.Linear(output_dim * 10, output_dim)
        # self.dropout = nn.Dropout(dropout)

        #---------------------Domain Classifier Network------------------------#
        # Fully-connected layer and Dropout
        self.domain_classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(len(filter_sizes) * n_filters, len(filter_sizes) * n_filters // 2),
            nn.ReLU(),
            nn.Linear(len(filter_sizes) * n_filters // 2, output_dim * 10),
            nn.ReLU(),
            nn.Linear(output_dim * 10, 2),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, text, alpha=1.0):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        x_feature = torch.cat(pooled, dim = 1)
        
        #x_feature = [batch size, n_filters * len(filter_sizes)]
        
        reverse_feature = GradientReversalFn.apply(x_feature, alpha)
        # print("reverse_feature",reverse_feature)
    
        regression_output = self.regression(x_feature)
    
        domain_classifier_output = self.domain_classifier(reverse_feature)


        return regression_output, domain_classifier_output

In [61]:
# INPUT_DIM = len(field_tweet.vocab) # these change for each model
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2, 3, 4, 5]
OUTPUT_DIM = 1
DROPOUT = 0.5
# PAD_IDX = field_tweet.vocab.stoi[field_tweet.pad_token] # these change for each model

### Model Architecture Creation for each variant, Loading pre-trained embeddings

In [66]:
dict_model_arch ={}
for name in list_name:
  
  # INPUT_DIM = len(field_tweet.vocab) # single model
  INPUT_DIM = len(dict_fields[name]['Tweet'][1].vocab)
  # print(INPUT_DIM)

  # PAD_IDX = field_tweet.vocab.stoi[field_tweet.pad_token] # # single model
  PAD_IDX = dict_fields[name]['Tweet'][1].vocab.stoi[dict_fields[name]['Tweet'][1].pad_token]
  # print(PAD_IDX)

  dict_model_arch[name] = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
  dict_model_arch[name].to(DEVICE)

  # pretrained_embeddings = field_tweet.vocab.vectors # single model
  pretrained_embeddings = dict_fields[name]['Tweet'][1].vocab.vectors

  # model.embedding.weight.data.copy_(pretrained_embeddings) # single model
  dict_model_arch[name].embedding.weight.data.copy_(pretrained_embeddings)

  # UNK_IDX = field_tweet.vocab.stoi[field_tweet.unk_token] # single model
  UNK_IDX = dict_fields[name]['Tweet'][1].vocab.stoi[dict_fields[name]['Tweet'][1].unk_token]

  # model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) # single model
  dict_model_arch[name].embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) 
  
  # model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM) # single model
  dict_model_arch[name].embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM) 

dict_model_arch

{'EI_anger': CNN1d(
   (embedding): Embedding(4824, 100, padding_idx=1)
   (convs): ModuleList(
     (0): Conv1d(100, 100, kernel_size=(2,), stride=(1,))
     (1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
     (2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
     (3): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
   )
   (regression): Sequential(
     (0): Dropout(p=0.5, inplace=False)
     (1): Linear(in_features=400, out_features=200, bias=True)
     (2): ReLU()
     (3): Linear(in_features=200, out_features=10, bias=True)
     (4): ReLU()
     (5): Linear(in_features=10, out_features=1, bias=True)
   )
   (domain_classifier): Sequential(
     (0): Dropout(p=0.5, inplace=False)
     (1): Linear(in_features=400, out_features=200, bias=True)
     (2): ReLU()
     (3): Linear(in_features=200, out_features=10, bias=True)
     (4): ReLU()
     (5): Linear(in_features=10, out_features=2, bias=True)
     (6): LogSoftmax(dim=1)
   )
 ), 'EI_sadness': CNN1d(
   (embedding): Embed

In [64]:
# model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
# model.to(DEVICE)

### Load Pre trained embeddings
we'll load the pre-trained *embeddings*

In [40]:
# pretrained_embeddings = field_tweet.vocab.vectors
# model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.9277, -0.7689,  0.5307,  ..., -0.7100, -0.4301, -0.2852],
        [ 0.0811, -1.0963, -0.0437,  ..., -0.5124,  0.0774, -0.3126],
        [ 0.3447,  0.7465,  0.6800,  ..., -1.8343,  1.1020,  0.7335],
        ...,
        [-0.3437, -0.1366, -0.6357,  ..., -0.4675,  1.4354, -1.3787],
        [ 2.0333, -1.2754, -0.3370,  ...,  0.6044,  1.2364, -0.1271],
        [-0.2258, -1.4275, -2.2880,  ..., -0.3692,  0.9410,  1.5251]])

In [41]:
# field_tweet.vocab.vectors.shape

torch.Size([4788, 100])

In [42]:
# UNK_IDX = field_tweet.vocab.stoi[field_tweet.unk_token]

# model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
# model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Training the model

### Without training one forward pass

In [ ]:
for name, model_arch in dict_model_arch.items():
  for batch in dict_iterator[name]['train_iterator']:
    print(batch.tweet)
    output = model_arch(batch.tweet)
    print (output)
    break

In [ ]:
# import torch.optim as optim

# optimizer = optim.Adam(model.parameters())

# criterion = nn.BCEWithLogitsLoss()

# model = model.to(DEVICE)
# criterion = criterion.to(DEVICE)

### Typical Train Model Function

In [70]:
# Typical Training Function

from tqdm import tqdm # for beautiful model training updates

def train_model(model, device, train_loader, optimizer, epoch):
    model.train() # setting the model in training mode
    pbar = tqdm(train_loader) # putting the iterator in pbara
    correct = 0 # for accuracy numerator
    processed =0 # for accuracy denominator
    epoch_loss = 0.0
    for batch_idx, batch in enumerate(pbar):

        tweets, intensities = batch.tweet.to(device), batch.intensity.to(device)  # plural, we are not interested in domain
        #sending data to CPU or GPU as per device

        optimizer.zero_grad() # setting gradients to zero to avoid accumulation

        y_preds,_ = model(tweets) # forward pass, result captured in y_preds (plural as there are many body in a batch)
        # we are not interested in domain prediction
        # the predictions are in one hot vector

        regression_loss = regression_loss_function(y_preds,intensities.unsqueeze(1)) # Computing loss
        # loss = F.mse_loss(y_preds,intensities.unsqueeze(1)) # Computing loss

        train_regresion_losses.append(regression_loss.item()) # to capture loss over many epochs

        regression_loss.backward() # backpropagation
        optimizer.step() # updating the params

        # preds = y_preds.argmax(dim=1, keepdim=True)  # get the index olf the max log-probability
        # correct += preds.eq(labels.view_as(preds)).sum().item()
        epoch_loss += regression_loss.item()

        processed += len(tweets)

        pbar.set_description(desc= f'Loss={regression_loss.item()} Batch_id={batch_idx} Epoch Average loss={100*epoch_loss/processed:0.4f}')
    train_accuracy.append(100*epoch_loss/len(train_loader))

### Typical Test Function

In [71]:
def test_model(model,device, data_loader, mode= 'test'):
    model.eval() # setting the model in evaluation mode
    loss = 0
    correct = 0 # for accuracy numerator
    test_regresion_losses =[] # for overall batches (summed over batches)
    valid_regresion_losses =[] # for overall batches (summed over batches)

    with torch.no_grad():
        for batch in data_loader:

            tweets, intensities  = batch.tweet.to(device), batch.intensity.to(device) #sending data to CPU or GPU as per device
            # we are not interested in domains
            
            y_preds,_ = model(tweets) # forward pass, result captured in outputs (plural as there are many bodies in a batch)
            # the outputs are in batch size x one hot vector 
            # not interested in domain output

            regression_loss = regression_loss_function(y_preds,intensities.unsqueeze(1))

            if mode == 'test':
              test_regresion_losses.append(regression_loss.item())
              # print(f'...in the batch...{regression_loss}')
            else:
              valid_regresion_losses.append(regression_loss.item())
              # print(f'...in the batch...{regression_loss}')

        # regression_loss.item() /= len(data_loader.dataset) # average test loss
        if mode == 'test':
          # total_test_regression_loss = sum(test_regresion_losses)
          # test_regresion_losses.append(regression_loss) # to capture loss over many batches
          # print('...Average test loss: {:.8f}'.format((total_test_regression_loss)/len(data_loader.dataset)))
          print(f'TEST LOSS (Average) : {sum(test_regresion_losses) / len(data_loader)}')
        else:
          # valid_regresion_losses.append(regression_loss) # to capture loss over many batches
          # total_valid_regression_loss = sum(valid_regresion_losses)
          # print('...Average validation loss: {:.8f}'.format((total_valid_regression_loss)/len(data_loader.dataset)))
          print(f'VALIDATION LOSS (Average) : {sum(valid_regresion_losses) / len(data_loader)}')

In [72]:
# EXECUTION (NON DANN) FOR MULTIPLE MODELS
lr = 2e-5
EPOCHS = 2
# EPOCHS = 100
dict_non_dann_model_saved= {}
for name, model_arch in dict_model_arch.items():
  model = model_arch
  optimizer = optim.Adam(model.parameters(), lr=lr)
  domain_loss_function= nn.BCEWithLogitsLoss()
  regression_loss_function = nn.L1Loss()
  model = model.to(DEVICE)
  domain_loss_function = domain_loss_function.to(DEVICE)
  regression_loss_function = regression_loss_function.to(DEVICE)

  # train_losses = [] # to capture train losses over training epochs
  train_accuracy = [] # to capture train accuracy over training epochs
  # val_losses = [] # to capture validation loss
  # test_losses = [] # to capture test losses 
  # test_accuracy = [] # to capture test accuracy 

  # dict_val_loss = {}
  # dict_test_loss = {}
  train_regresion_losses = [] # to capture train losses over training epochs
  train_domain_losses = []
  train_accuracy = [] # to capture train accuracy over training epochs
  # valid_regresion_losses = [] # to capture validation loss
  # test_regresion_losses = [] # to capture test losses 
  total_test_regression_loss =[]
  total_valid_regression_loss =[]
  print(f'----------------------training started for {name}-----------------')
  for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    # train_model(model, DEVICE, train_iterator, optimizer, epoch) # single model
    train_model(model, DEVICE, dict_iterator[name]['train_iterator'], optimizer, epoch)
    
    # test_model(model, DEVICE, valid_iterator, mode = 'val')# single model
    test_model(model, DEVICE, dict_iterator[name]['val_iterator'], mode = 'val')

    # test_model(model, DEVICE, test_iterator, mode = 'test')# single model
    test_model(model, DEVICE, dict_iterator[name]['test_iterator'], mode = 'test')

  # dict_val_loss[name] = val_losses
  # dict_test_loss[name] = test_losses

  model_name = name + "_non_dann"+".pt"
  torch.save(model.state_dict(), os.path.join(MODEL_DIR, model_name))
  dict_non_dann_model_saved[name]= model_name
  print(f'----------------------training complete for {name}-----------------')
# print(dict_val_loss.items())
# print(dict_test_loss.items())

----------------------training started for EI_anger-----------------
EPOCH: 1


Loss=0.14859448373317719 Batch_id=212 Epoch Average loss=2.6427: 100%|██████████| 213/213 [00:09<00:00, 21.72it/s]


VALIDATION LOSS (Average) : 0.16344887018203735
TEST LOSS (Average) : 0.15497437119483948
EPOCH: 2


Loss=0.18956522643566132 Batch_id=212 Epoch Average loss=1.7581: 100%|██████████| 213/213 [00:07<00:00, 29.36it/s]


VALIDATION LOSS (Average) : 0.16332216560840607
TEST LOSS (Average) : 0.15504890183607736
----------------------training complete for EI_anger-----------------
----------------------training started for EI_sadness-----------------
EPOCH: 1


...

VALIDATION LOSS (Average) : 0.15762850642204285
TEST LOSS (Average) : 0.15292058885097504
----------------------training complete for EI_joy-----------------
----------------------training started for V-----------------
EPOCH: 1


Loss=0.20809084177017212 Batch_id=147 Epoch Average loss=3.7406: 100%|██████████| 148/148 [00:06<00:00, 23.61it/s]


VALIDATION LOSS (Average) : 0.20140859484672546
TEST LOSS (Average) : 0.17838483055432638
EPOCH: 2


Loss=0.2336803376674652 Batch_id=147 Epoch Average loss=2.4642: 100%|██████████| 148/148 [00:05<00:00, 28.41it/s]


VALIDATION LOSS (Average) : 0.19873183965682983
TEST LOSS (Average) : 0.17536505063374838
----------------------training complete for V-----------------


In [ ]:
# # EXECUTION

# lr = 2e-5
# optimizer = optim.Adam(model.parameters(), lr=lr)
# domain_loss_function= nn.BCEWithLogitsLoss()
# regression_loss_function = nn.L1Loss()


# model = model.to(DEVICE)
# domain_loss_function = domain_loss_function.to(DEVICE)
# regression_loss_function = regression_loss_function.to(DEVICE)

# # train_losses = [] # to capture train losses over training epochs
# train_accuracy = [] # to capture train accuracy over training epochs
# # val_losses = [] # to capture validation loss
# # test_losses = [] # to capture test losses 
# # test_accuracy = [] # to capture test accuracy 

# # EPOCHS = 2
# EPOCHS = 100
# # dict_val_loss = {}
# # dict_test_loss = {}


# train_regresion_losses = [] # to capture train losses over training epochs
# train_domain_losses = []
# train_accuracy = [] # to capture train accuracy over training epochs
# # valid_regresion_losses = [] # to capture validation loss
# # test_regresion_losses = [] # to capture test losses 
# total_test_regression_loss =[]
# total_valid_regression_loss =[]
# # print(f'----------------------training started for {name}-----------------')
# for epoch in range(EPOCHS):
#   print("EPOCH:", epoch+1)
#   train_model(model, DEVICE, train_iterator, optimizer, epoch)
#   # print("for validation.......")
#   # val_name = train_name.replace("train", "val" )
#   # test_model(typical_model, device, dict_val_loader[val_name], mode = 'val')
#   test_model(model, DEVICE, valid_iterator, mode = 'val')


#   # print("for test  .......")
#   # test_name = train_name.replace("train", "test" )
#   # test_model(typical_model, device, dict_test_loader[test_name], mode = 'test')
#   test_model(model, DEVICE, test_iterator, mode = 'test')

# # dict_val_loss[name] = val_losses
# # dict_test_loss[name] = test_losses

# model_name = "Non_DANN"+".pt"
# torch.save(model.state_dict(), os.path.join(MODEL_DIR, model_name))
# # print(f'----------------------training complete for {name}-----------------')
# # print(dict_val_loss.items())
# # print(dict_test_loss.items())

EPOCH: 1


Loss=0.10554364323616028 Batch_id=201 Epoch Average loss=2.1569: 100%|██████████| 202/202 [00:01<00:00, 120.95it/s]


for validation.......
...Average batch validation loss is 0.1546570360660553
for test  .......
...Average batch test loss is 0.14828822761774063
EPOCH: 2


Loss=0.13246333599090576 Batch_id=201 Epoch Average loss=1.8359: 100%|██████████| 202/202 [00:01<00:00, 128.24it/s]


for validation.......
...Average batch validation loss is 0.13858754932880402
for test  .......
...Average batch test loss is 0.13643446005880833
EPOCH: 3


...

for validation.......
...Average batch validation loss is 0.13651412725448608
for test  .......
...Average batch test loss is 0.12829741649329662
EPOCH: 99


Loss=0.033625997602939606 Batch_id=201 Epoch Average loss=0.3309: 100%|██████████| 202/202 [00:02<00:00, 100.74it/s]


for validation.......
...Average batch validation loss is 0.13567854464054108
for test  .......
...Average batch test loss is 0.12754332087934017
EPOCH: 100


Loss=0.03215210884809494 Batch_id=201 Epoch Average loss=0.3229: 100%|██████████| 202/202 [00:01<00:00, 102.61it/s]


for validation.......
...Average batch validation loss is 0.13678020238876343
for test  .......
...Average batch test loss is 0.128689618781209


## DANN Model - Training and Testing

### Training

In [74]:
def compute_accuracy(logits, labels):
    
    predicted_labels_dict = {
      0: 0,
      1: 0,
    }
    
    predicted_label = logits.max(dim = 1)[1]
    
    for pred in predicted_label:
        predicted_labels_dict[pred.item()] += 1
    acc = (predicted_label == labels).float().mean()
    
    return acc, predicted_labels_dict

In [ ]:
# def binary_acc(y_pred, y_test):
#     y_pred_tag = torch.round(torch.sigmoid(y_pred))

#     correct_results_sum = (y_pred_tag == y_test).sum().float()
#     acc = correct_results_sum/y_test.shape[0]
#     acc = torch.round(acc * 100)
    
#     return acc

In [ ]:
# def evaluate(model, dataloader, mode = 'test', percentage = 5):
#     with torch.no_grad():
#         predicted_labels_dict = {                                                   
#           0: 0,                                                                     
#           1: 0,                                                                     
#         }
        
#         mean_accuracy = 0.0
#         # total_batches = len(dataloader)
#         # print("total_batches: ",total_batches )

        



#             sentiment_pred, _ = model(**inputs)
#             accuracy, predicted_labels = compute_accuracy(sentiment_pred, inputs["labels"])
#             mean_accuracy += accuracy
#             predicted_labels_dict[0] += predicted_labels[0]
#             predicted_labels_dict[1] += predicted_labels[1]  
#         print(predicted_labels_dict)
#     return mean_accuracy/total_batches

In [75]:
## DANN For multiple datasets
# n_epochs = 100 # number of epochs
n_epochs = 2 # number of epochs
lr = 2e-5

dict_dann_model_saved= {}
for name, model_arch in dict_model_arch.items():
  model = model_arch
  optimizer = optim.Adam(model.parameters(), lr=lr)
  model = model.to(DEVICE)
  domain_loss_function= nn.NLLLoss()
  regression_loss_function = nn.L1Loss()
  domain_loss_function = domain_loss_function.to(DEVICE)
  regression_loss_function = regression_loss_function.to(DEVICE)
  max_batches = min(len(dict_iterator[name]['train_iterator']), len(dict_target_iterator[name]))
  # max_batches = min(len(train_iterator), len(target_data)//TARGET_BATCH_SIZE)
  # max_batches = min(len(train_iterator), len(target_iterator))

  # print(max_batches)
  print(f'----------------------training started for DANN model - {name}-----------------')
  for epoch_idx in range(n_epochs):
      # source_iterator = iter(train_iterator) #single dataset
      source_iterator = iter(dict_iterator[name]['train_iterator'])
      # target_iterator = iter(target_iterator) #single dataset
      target_iterator = iter(dict_target_iterator[name]) 

      for batch_idx in range(max_batches):
          
          p = float(batch_idx + epoch_idx * max_batches) / (n_epochs * max_batches)
          alpha = 2. / (1. + np.exp(-10 * p)) - 1
          alpha = torch.tensor(alpha)
          
          model.train()          
          optimizer.zero_grad()

          ## SOURCE DATASET TRAINING UPDATE
          
          source_batch = next(source_iterator)
          source_tweets, source_intensities = source_batch.tweet.to(DEVICE), source_batch.intensity.to(DEVICE)  # plural, we are not interested in domain
          
          source_intensity_outputs, source_domain_outputs = model(source_tweets, alpha = alpha)

          loss_source_regression= regression_loss_function(source_intensity_outputs,source_intensities.unsqueeze(1)) # Computing regression loss

          source_domain_inputs = torch.zeros(len(source_batch), dtype=torch.long).to(DEVICE) # source domain has 0 id
          loss_source_domain = domain_loss_function(source_domain_outputs,source_domain_inputs)


          ## TARGET DATASET TRAINING UPDATE
          target_batch = next(iter(target_iterator))
          target_tweets= target_batch.tweet.to(DEVICE) # plural

          _, target_domain_outputs = model(target_tweets, alpha = alpha)

          target_domain_inputs = torch.ones(len(target_batch), dtype=torch.long).to(DEVICE) # target domain has 1 id
          loss_target_domain = domain_loss_function(target_domain_outputs,target_domain_inputs)

          # COMBINING LOSS
          loss = loss_source_regression + loss_source_domain + loss_target_domain
          loss.backward()
          optimizer.step()

          if (batch_idx % 100 == 0):
            print("Epoch [{}/{}] Step [{}/{}]: domain_loss_target={:.4f} / domain_loss_source={:.4f} / regression_loss_source={:.4f} / alpha={:.4f}"
                .format(epoch_idx + 1,
                        n_epochs,
                        batch_idx + 1,
                        max_batches,
                        loss_target_domain.item()
                        ,loss_source_domain.item()
                        ,loss_source_regression.item(),alpha))


      # Evaluate the model after every epoch

      # test_model(model, DEVICE, valid_iterator, mode = 'val') # single model
      test_model(model, DEVICE, dict_iterator[name]['val_iterator'], mode = 'val') 

      # test_model(model, DEVICE, test_iterator, mode = 'test') # single model
      test_model(model, DEVICE, dict_iterator[name]['test_iterator'], mode = 'test')

  model_name = name + "_dann.pt"
  torch.save(model.state_dict(), os.path.join(MODEL_DIR,model_name))
  dict_dann_model_saved[name] = model_name
  print(f'----------------------training complete for DANN model - {name}-----------------')


213
----------------------training started for DANN model - EI_anger-----------------
Epoch [1/2] Step [1/213]: domain_loss_target=0.7668 / domain_loss_source=0.6351 / regression_loss_source=0.0853 / alpha=0.0000
Epoch [1/2] Step [101/213]: domain_loss_target=0.8223 / domain_loss_source=0.5324 / regression_loss_source=0.1166 / alpha=0.8255
Epoch [1/2] Step [201/213]: domain_loss_target=0.8631 / domain_loss_source=0.5346 / regression_loss_source=0.1085 / alpha=0.9819
VALIDATION LOSS (Average) : 0.16267548501491547
TEST LOSS (Average) : 0.15399124721686044
Epoch [2/2] Step [1/213]: domain_loss_target=0.8655 / domain_loss_source=0.5222 / regression_loss_source=0.1549 / alpha=0.9866
Epoch [2/2] Step [101/213]: domain_loss_target=0.8745 / domain_loss_source=0.4896 / regression_loss_source=0.1825 / alpha=0.9987
Epoch [2/2] Step [201/213]: domain_loss_target=0.8989 / domain_loss_source=0.5315 / regression_loss_source=0.1699 / alpha=0.9999
VALIDATION LOSS (Average) : 0.16253596544265747
TEST L

In [ ]:
#  ## DANN training function attempt 2 with NLLLoss
# # n_epochs = 100 # number of epochs
# n_epochs = 2 # number of epochs
# lr = 2e-5


# optimizer = optim.Adam(model.parameters(), lr=lr)

# # loss_fn_sentiment_regression = torch.nn.NLLLoss()
# # loss_fn_domain_classifier = torch.nn.NLLLoss()

# model = model.to(DEVICE)
# domain_loss_function= nn.NLLLoss()
# regression_loss_function = nn.L1Loss()
# domain_loss_function = domain_loss_function.to(DEVICE)
# regression_loss_function = regression_loss_function.to(DEVICE)

# max_batches = min(len(train_iterator), len(target_data)//TARGET_BATCH_SIZE)
# # max_batches = min(len(train_iterator), len(target_iterator))

# print(max_batches)

# for epoch_idx in range(n_epochs):
    
#     source_iterator = iter(train_iterator)
#     target_iterator = iter(target_iterator)

#     for batch_idx in range(max_batches):
        
#         p = float(batch_idx + epoch_idx * max_batches) / (n_epochs * max_batches)
#         alpha = 2. / (1. + np.exp(-10 * p)) - 1
#         alpha = torch.tensor(alpha)
        
#         model.train()
        
#         # if(batch_idx%100 == 0 ):
#         #     print("Training Step:", batch_idx)
        
#         optimizer.zero_grad()

#         ## SOURCE DATASET TRAINING UPDATE
        
#         source_batch = next(source_iterator)
#         source_tweets, source_intensities = source_batch.tweet.to(DEVICE), source_batch.intensity.to(DEVICE)  # plural, we are not interested in domain
        
#         source_intensity_outputs, source_domain_outputs = model(source_tweets, alpha = alpha)

#         loss_source_regression= regression_loss_function(source_intensity_outputs,source_intensities.unsqueeze(1)) # Computing regression loss

#         source_domain_inputs = torch.zeros(len(source_batch), dtype=torch.long).to(DEVICE) # source domain has 0 id
#         loss_source_domain = domain_loss_function(source_domain_outputs,source_domain_inputs)


#         ## TARGET DATASET TRAINING UPDATE
#         target_batch = next(iter(target_iterator))
#         target_tweets= target_batch.tweet.to(DEVICE) # plural

#         _, target_domain_outputs = model(target_tweets, alpha = alpha)

#         target_domain_inputs = torch.ones(len(target_batch), dtype=torch.long).to(DEVICE) # target domain has 1 id
#         loss_target_domain = domain_loss_function(target_domain_outputs,target_domain_inputs)

#         # COMBINING LOSS
#         loss = loss_source_regression + loss_source_domain + loss_target_domain
#         loss.backward()
#         optimizer.step()

#         if (batch_idx % 100 == 0):
#           print("Epoch [{}/{}] Step [{}/{}]: domain_loss_target={:.4f} / domain_loss_source={:.4f} / regression_loss_source={:.4f} / alpha={:.4f}"
#               .format(epoch_idx + 1,
#                       n_epochs,
#                       batch_idx + 1,
#                       max_batches,
#                       loss_target_domain.item()
#                       ,loss_source_domain.item()
#                       ,loss_source_regression.item(),alpha))


#     # Evaluate the model after every epoch


#     print("for validation.......")
#     test_model(model, DEVICE, valid_iterator, mode = 'val')
#     print("for test  .......")
#     test_model(model, DEVICE, test_iterator, mode = 'test')

  
# torch.save(model.state_dict(), os.path.join(MODEL_DIR, "epoch_" + str(epoch_idx)  +  ".pt" ))


# Equality Evaluation using Equity Evaluation Corpus

## Loading Evaluation Data
format `[ID	Sentence	Template	Person	Gender	Race Emotion	Emotion word]`

In [76]:
data_EEC = TASK1.EEC['eec']
df_EEC = pd.read_csv(data_EEC)
df_EEC.head()

,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


## Creating evalution function (includes pre-processing)

In [77]:
## padding function : adds padding / truncates to max size
def pad_or_truncate(some_list, target_len = MAX_SIZE ,pad_idx = PAD_IDX):
    return some_list[:target_len] + [pad_idx]*(target_len - len(some_list))

## preprocessing function, takes in a tweet and returns padded indexed tweet (input for model)
def text_pipeline(tweet):
    indexed_tweet = [field_tweet.vocab.__getitem__(token) for token in preprocess_tweet(tweet)]
    # print(indexed_tweet)
    return pad_or_truncate(indexed_tweet, MAX_SIZE , pad_idx = PAD_IDX)
    # print(indexed_tweet_padded)

In [78]:
# i = random.randint(0,len(df_EEC))
# tweet_example = df_EEC['Sentence'][i]
# print(tweet_example, text_pipeline(tweet_example))

I made my husband feel enraged. [7, 286, 23, 2975, 116, 0, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## Loading model

In [ ]:
### Loading Model


# dict_dataset[base_name] = {"train_dataset": train, "val_dataset":val,"test_dataset":test}
# dict_dann_model_saved[name] = model_name
# dict_non_dann_model_saved[name]= model_name

dict_loaded_models = {}
for name in list_name:
  non_dann_model_name = dict_non_dann_model_saved[name]
  dann_model_name = dict_dann_model_saved[name]
  # print(non_dann_model_name,dann_model_name)
  
  INPUT_DIM = len(dict_fields[name]['Tweet'][1].vocab)
  PAD_IDX = dict_fields[name]['Tweet'][1].vocab.stoi[dict_fields[name]['Tweet'][1].pad_token]

  loaded_model_non_dann = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
  loaded_model_non_dann.load_state_dict(torch.load(os.path.join(MODEL_DIR, non_dann_model_name),map_location=torch.device(DEVICE)))
  loaded_model_non_dann.eval()

  loaded_model_dann = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
  loaded_model_dann.load_state_dict(torch.load(os.path.join(MODEL_DIR, dann_model_name),map_location=torch.device(DEVICE)))
  loaded_model_dann.eval()

  dict_loaded_models[name]={"non_dann":loaded_model_non_dann,"dann":loaded_model_dann}

print(dict_loaded_models)

In [ ]:
### Loading Model (single dataset)

# dict_model_name = {'non_dann':'Non_DANN.pt','dann':'epoch_99.pt'}
# dict_loaded_model ={}
# for model_type, model_name in dict_model_name.items():
#   loaded_model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
#   loaded_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, model_name),map_location=torch.device(DEVICE)))
#   loaded_model.eval()
#   dict_loaded_model[model_type] = loaded_model
# print(dict_loaded_model)

In [93]:
from torch.cuda import Device
def predict(tweet, model, text_pipeline,device = DEVICE):
  with torch.no_grad():
    tweet_tensor = torch.tensor(text_pipeline(tweet)).unsqueeze(0).to(device)
    output = model(tweet_tensor)
    return output[0].item()

In [84]:
# i = random.randint(0,len(df_EEC))
# tweet_example = df_EEC['Sentence'][i]
# loaded_model_device = 'cpu'
# loaded_model = dict_loaded_models['EI_anger']['dann'].to(loaded_model_device)
# print(predict(tweet_example, loaded_model,text_pipeline, device= loaded_model_device))

0.43954771757125854


## Creating Sentence pairs (as per SEMVAL18 paper)

In [94]:
dict_f_m_noun_phrase = {'she':'he', 
            'her':'him',
            'this woman':'this man',
            'this girl':'this boy',
            'my sister' : 'my brother',
            'my daughter' : 'my son',
            'my wife': 'my husband',
            'my girlfriend':'my boyfriend',
            'my mother':'my father',
            'my aunt':'my uncle',
            'my mom': 'my dad'
            }

name_male = ['Alonzo','Jamel','Alphonse','Jerome','Leroy','Torrance','Darnell','Lamar','Malik','Terrence','Adam','Harry','Josh','Roger','Alan','Frank','Justin','Ryan','Andrew','Jack'] 
name_female = ['Nichelle','Shereen','Ebony','Latisha','Shaniqua','Jasmine','Tanisha','Tia','Lakisha','Latoya','Amanda','Courtney','Heather','Melanie','Katie','Betsy','Kristin','Nancy','Stephanie','Ellen']


In [95]:
list_unique_template = list(df_EEC['Template'].dropna().unique())
# print(list_unique_template)
list_emotion_word = list(df_EEC['Emotion word'].unique()) # contains nan also
# print(list_emotion_word)
list_gender = list(df_EEC['Gender'].dropna().unique())
# print(list_gender)
list_person = list(df_EEC['Person'].unique())   
# print(list_person)

In [56]:
# list_f_m_noun_phrase =[]
# list_f_m_noun_phrase.extend(name_male)
# list_f_m_noun_phrase.extend(name_female)
# [list_f_m_noun_phrase.extend([f,m]) for f,m in dict_f_m_noun_phrase.items()]
# print(list_f_m_noun_phrase)
# assert set(list_f_m_noun_phrase)<= set(list_person), "The noun phrases are not subset of overall person list"

In [87]:
print(list_emotion_word)
# list_emotion_word= list_emotion_word.append('')
# print(list_emotion_word)

['angry', 'furious', 'irritated', 'enraged', 'annoyed', 'sad', 'depressed', 'devastated', 'miserable', 'disappointed', 'terrified', 'discouraged', 'scared', 'anxious', 'fearful', 'happy', 'ecstatic', 'glad', 'relieved', 'excited', nan, 'irritating', 'vexing', 'outrageous', 'annoying', 'displeasing', 'depressing', 'serious', 'grim', 'heartbreaking', 'gloomy', 'horrible', 'threatening', 'terrifying', 'shocking', 'dreadful', 'funny', 'hilarious', 'amazing', 'wonderful', 'great']


In [88]:
# Template - F - M Noun Phrases chunks
dict_noun_phrase_sentence_pair = {}
## take a subset where Race field is not populated
df_noun_phrase_subset=  df_EEC[df_EEC['Race'].isna()] ## includes values which do not have Race 
count = 0
# print(len(df_noun_phrase_subset),df_noun_phrase_subset.head())
for template in list_unique_template:
  for emotion_word in list_emotion_word:
    condition_1 = (df_noun_phrase_subset['Template']== template)
    condition_2 = (df_noun_phrase_subset['Emotion word'] == emotion_word)
    condition_3 = (df_noun_phrase_subset['Gender'] == 'female')
    condition_4 = (df_noun_phrase_subset['Gender'] == 'male')
    list_female = df_noun_phrase_subset[(condition_1) & (condition_2) & (condition_3)]['Sentence'].to_list()
    list_male = df_noun_phrase_subset[(condition_1) & (condition_2) & (condition_4)]['Sentence'].to_list()
    if ((len(list_female) >0) & (len(list_male) >0)):
      dict_noun_phrase_sentence_pair[count]=(list_female,list_male)
      # print(emotion_word, "emotion word")
      count = count + 1
    
    if pd.isnull(emotion_word):
      condition_5 = (df_noun_phrase_subset['Emotion word'].isna())
      list_female_2 = df_noun_phrase_subset[(condition_5) & (condition_1) & (condition_3)]['Sentence'].to_list()
      list_male_2 = df_noun_phrase_subset[(condition_5) & (condition_1) & (condition_4)]['Sentence'].to_list()
      if ((len(list_female_2) >0) & (len(list_male_2) >0)):
        dict_noun_phrase_sentence_pair[count]=(list_female_2,list_male_2)
        # print("no emotion word")
        count = count + 1

print(count)
print(len(dict_noun_phrase_sentence_pair),"dict_noun_phrase_sentence_pair............ \n",dict_noun_phrase_sentence_pair)


144
144 \dict_noun_phrase_sentence_pair............ 
 {0: (['She feels angry.', 'This woman feels angry.', 'This girl feels angry.', 'My sister feels angry.', 'My daughter feels angry.', 'My wife feels angry.', 'My girlfriend feels angry.', 'My mother feels angry.', 'My aunt feels angry.', 'My mom feels angry.'], ['He feels angry.', 'This man feels angry.', 'This boy feels angry.', 'My brother feels angry.', 'My son feels angry.', 'My husband feels angry.', 'My boyfriend feels angry.', 'My father feels angry.', 'My uncle feels angry.', 'My dad feels angry.']), 1: (['She feels furious.', 'This woman feels furious.', 'This girl feels furious.', 'My sister feels furious.', 'My daughter feels furious.', 'My wife feels furious.', 'My girlfriend feels furious.', 'My mother feels furious.', 'My aunt feels furious.', 'My mom feels furious.'], ['He feels furious.', 'This man feels furious.', 'This boy feels furious.', 'My brother feels furious.', 'My son feels furious.', 'My husband feels furio

In [133]:
# Template - F - M Noun Phrases chunks ORIGINAL
dict_original_sentence_pair = {}
count = 0

for template in list_unique_template:
  for f, m in dict_f_m_noun_phrase.items():
    condition_1 = df_EEC['Template']== template
    condition_2 = df_EEC['Person']== f
    condition_3 = df_EEC['Person']== m
    df_temp_f = df_EEC[(condition_1 & condition_2 )] 
    df_temp_m = df_EEC[(condition_1 & condition_3 )]
    for emotion_word in list_emotion_word:
      
      condition_4 = df_EEC['Emotion word'] == emotion_word
      
      k = df_temp_f[condition_4]['Sentence']
      v = df_temp_m[condition_4]['Sentence']
      assert len(k)==len(v), "Problem is in Noun Phase Chunks where emotion_word is not null"
      if len(k) > 0 and len (v) > 0:
        dict_original_sentence_pair[count] = (k.values[0],v.values[0])
        count = count + 1
      
      ## Checking for column values where emotion word value blank
      if pd.isnull(emotion_word):
        k_null = df_temp_f[df_temp_f['Emotion word'].isna()]['Sentence']
        v_null = df_temp_m[df_temp_m['Emotion word'].isna()]['Sentence']
        assert len(k_null)==len(v_null), "Problem is in Noun Phase Chunks where emotion_word is  null"
        if len(k_null) > 0 and len (v_null) > 0:
          dict_original_sentence_pair[count] = (k_null.values[0],v_null.values[0])
          count = count + 1
      
print(len(dict_original_sentence_pair),"dict_original_sentence_pair............ \n",dict_original_sentence_pair)

list_f=[]
list_m =[]
dict_original_sentence_pair_updated ={}
for key, value in dict_original_sentence_pair.items():
  list_f.append(value[0])
  list_m.append(value[1])

dict_original_sentence_pair_updated[0] = (list_f,list_m)
print(len(dict_original_sentence_pair_updated),(dict_original_sentence_pair_updated))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


1440 dict_original_sentence_pair............ 
 {0: ('She feels angry.', 'He feels angry.'), 1: ('She feels furious.', 'He feels furious.'), 2: ('She feels irritated.', 'He feels irritated.'), 3: ('She feels enraged.', 'He feels enraged.'), 4: ('She feels annoyed.', 'He feels annoyed.'), 5: ('She feels sad.', 'He feels sad.'), 6: ('She feels depressed.', 'He feels depressed.'), 7: ('She feels devastated.', 'He feels devastated.'), 8: ('She feels miserable.', 'He feels miserable.'), 9: ('She feels disappointed.', 'He feels disappointed.'), 10: ('She feels terrified.', 'He feels terrified.'), 11: ('She feels discouraged.', 'He feels discouraged.'), 12: ('She feels scared.', 'He feels scared.'), 13: ('She feels anxious.', 'He feels anxious.'), 14: ('She feels fearful.', 'He feels fearful.'), 15: ('She feels happy.', 'He feels happy.'), 16: ('She feels ecstatic.', 'He feels ecstatic.'), 17: ('She feels glad.', 'He feels glad.'), 18: ('She feels relieved.', 'He feels relieved.'), 19: ('She f

In [96]:
# for Named people

dict_list_named_sentence_pairs ={}
df_EEC_subset = df_EEC.dropna(subset = ['Race']) ## removes values which do not have Race 
print(len(df_EEC_subset))

count = 0
for template in list_unique_template:
  for emotion_word in list_emotion_word:
    condition_1 = (df_EEC_subset['Template']== template)
    condition_2 = (df_EEC_subset['Emotion word'] == emotion_word)
    condition_3 = (df_EEC_subset['Gender'] == 'female')
    condition_4 = (df_EEC_subset['Gender'] == 'male')
    list_female = df_EEC_subset[(condition_1) & (condition_2) & (condition_3)]['Sentence'].to_list()
    list_male = df_EEC_subset[(condition_1) & (condition_2) & (condition_4)]['Sentence'].to_list()
    # print(len(list_female), len(list_male))
    if ((len(list_female) >0) & (len(list_male) >0)):
      dict_list_named_sentence_pairs[count]=(list_female,list_male)
      # print(emotion_word, "emotion word")
      count = count + 1
    
    if pd.isnull(emotion_word):
      condition_5 = (df_EEC_subset['Emotion word'].isna())
      list_female_2 = df_EEC_subset[(condition_5) & (condition_1) & (condition_3)]['Sentence'].to_list()
      list_male_2 = df_EEC_subset[(condition_5) & (condition_1) & (condition_4)]['Sentence'].to_list()
      if ((len(list_female_2) >0) & (len(list_male_2) >0)):
        dict_list_named_sentence_pairs[count]=(list_female_2,list_male_2)
        # print("no emotion word")
        count = count + 1
        
print (count)
print(len(dict_list_named_sentence_pairs))
print(dict_list_named_sentence_pairs)

5760
144
144
{0: (['Nichelle feels angry.', 'Shereen feels angry.', 'Ebony feels angry.', 'Latisha feels angry.', 'Shaniqua feels angry.', 'Jasmine feels angry.', 'Tanisha feels angry.', 'Tia feels angry.', 'Lakisha feels angry.', 'Latoya feels angry.', 'Amanda feels angry.', 'Courtney feels angry.', 'Heather feels angry.', 'Melanie feels angry.', 'Katie feels angry.', 'Betsy feels angry.', 'Kristin feels angry.', 'Nancy feels angry.', 'Stephanie feels angry.', 'Ellen feels angry.'], ['Alonzo feels angry.', 'Jamel feels angry.', 'Alphonse feels angry.', 'Jerome feels angry.', 'Leroy feels angry.', 'Torrance feels angry.', 'Darnell feels angry.', 'Lamar feels angry.', 'Malik feels angry.', 'Terrence feels angry.', 'Adam feels angry.', 'Harry feels angry.', 'Josh feels angry.', 'Roger feels angry.', 'Alan feels angry.', 'Frank feels angry.', 'Justin feels angry.', 'Ryan feels angry.', 'Andrew feels angry.', 'Jack feels angry.']), 1: (['Nichelle feels furious.', 'Shereen feels furious.', 

In [91]:
print(dict_list_named_sentence_pairs[0][0],"\n",dict_list_named_sentence_pairs[0][1])

['Nichelle feels angry.', 'Shereen feels angry.', 'Ebony feels angry.', 'Latisha feels angry.', 'Shaniqua feels angry.', 'Jasmine feels angry.', 'Tanisha feels angry.', 'Tia feels angry.', 'Lakisha feels angry.', 'Latoya feels angry.', 'Amanda feels angry.', 'Courtney feels angry.', 'Heather feels angry.', 'Melanie feels angry.', 'Katie feels angry.', 'Betsy feels angry.', 'Kristin feels angry.', 'Nancy feels angry.', 'Stephanie feels angry.', 'Ellen feels angry.'] 
 ['Alonzo feels angry.', 'Jamel feels angry.', 'Alphonse feels angry.', 'Jerome feels angry.', 'Leroy feels angry.', 'Torrance feels angry.', 'Darnell feels angry.', 'Lamar feels angry.', 'Malik feels angry.', 'Terrence feels angry.', 'Adam feels angry.', 'Harry feels angry.', 'Josh feels angry.', 'Roger feels angry.', 'Alan feels angry.', 'Frank feels angry.', 'Justin feels angry.', 'Ryan feels angry.', 'Andrew feels angry.', 'Jack feels angry.']


## Two Sample t- test

In [79]:
# f ='She feels angry.'
# m ='He feels angry.'
# f_indices = text_pipeline(f)
# m_indices = text_pipeline(m)
# f_value = predict(f, loaded_model,text_pipeline,device= loaded_model_device)
# m_value = predict(m, loaded_model,text_pipeline,device= loaded_model_device)
# print(f_value,m_value)
# stats.ttest_rel(f_value, m_value)

0.32102489471435547 0.33261770009994507


Ttest_relResult(statistic=nan, pvalue=nan)

In [136]:
# Function for t-test processing

def two_sample_test(dict_sentence_pairs ={}, text_pipeline = text_pipeline, loaded_model= None, loaded_model_device = 'cpu')-> dict:
  assert loaded_model is not None, "No Model Selected for t-test"
  dict_t_test_result_sentence_pair ={}
  for key, value in dict_sentence_pairs.items():

    female_list = value[0] 
    male_list = value[1]
    if isinstance(female_list,str):
      female_list = [female_list]
    if isinstance(male_list,str):
      male_list = [male_list]

    assert len(female_list) == len(male_list), f"Different lengths: Lengths of female list is {len(female_list)} and male list is {len(male_list)}"
    
    female_list_indices = [text_pipeline(tweet_example) for tweet_example in female_list]
    male_list_indices = [text_pipeline(tweet_example)for tweet_example in male_list]

    female_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in female_list]
    male_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in male_list]

    t_test_result = stats.ttest_rel(female_list_output, male_list_output)
    dict_t_test_result_sentence_pair[key] = (t_test_result.statistic, t_test_result.pvalue,mean(female_list_output)-mean(male_list_output))
  # print(dict_t_test_result_sentence_pair)
  return dict_t_test_result_sentence_pair



In [147]:
# dict_loaded_models[name]={"non_dann":loaded_model_non_dann,"dann":loaded_model_dann}

dict_sentence_pairs = {'named': dict_list_named_sentence_pairs ,
                       'noun_phrase': dict_noun_phrase_sentence_pair,
                       'original_noun_phrase':dict_original_sentence_pair_updated}
dict_t_test = {}
for name, model_dict in dict_loaded_models.items():
  dict_t_test_level_1 = {}
  for model_type, model in model_dict.items():
    dict_t_test_level_2 ={}
    for sentence_pair_name, dict_sentence_pair in dict_sentence_pairs.items():
      # key_name = str(name+ "_" + model_type + "_" + sentence_pair_name)
      # print(key_name)
      loaded_model = dict_loaded_models[name][model_type]
      # dict_t_test[key_name] = two_sample_test(dict_sentence_pairs = dict_sentence_pair ,
      #                                         text_pipeline = text_pipeline, 
      #                                         loaded_model = loaded_model, 
      #                                         loaded_model_device = 'cpu')
      dict_t_test_level_2[sentence_pair_name] = two_sample_test(dict_sentence_pairs = dict_sentence_pair ,
                                        text_pipeline = text_pipeline, 
                                        loaded_model = loaded_model, 
                                        loaded_model_device = 'cpu')
      
    dict_t_test_level_1[model_type] = dict_t_test_level_2
    
  dict_t_test[name] = dict_t_test_level_1
  
print(dict_t_test)
# for model_type, loaded_model in dict_loaded_model.items():
#   dict_t_test[str(model_type)+"_noun_phrase"] = two_sample_test(dict_sentence_pairs =dict_noun_phrase_sentence_pair,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')
#   dict_t_test[str(model_type)+"_named"] = two_sample_test(dict_sentence_pairs =dict_list_named_sentence_pairs,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')


{'EI_anger': {'non_dann': {'named': {0: (-1.9246552985895022, 0.06937894308335793, -0.0013516709208488131), 1: (-2.008530111493965, 0.059015256511744524, -0.001458911597728707), 2: (-1.8951536736010397, 0.07339128407247694, -0.00140501409769056), 3: (-1.8951536736010397, 0.07339128407247694, -0.00140501409769056), 4: (-2.1779245152351754, 0.04221540668509497, -0.0015045315027236716), 5: (-2.648075838714515, 0.015866249147433193, -0.0017042383551597595), 6: (-2.1018474681679837, 0.04913425936739607, -0.0012712687253951804), 7: (-1.8951536736010397, 0.07339128407247694, -0.00140501409769056), 8: (-1.8951536736010397, 0.07339128407247694, -0.00140501409769056), 9: (-1.460569830262038, 0.16047320249718272, -0.0006277486681938171), 10: (-2.054308026113983, 0.05396415105982764, -0.0017071768641471752), 11: (-1.6818289709495244, 0.10896545901924828, -0.001728479564189922), 12: (-1.6598951151128283, 0.11335043038137606, -0.0012698620557785034), 13: (-1.7147530605800843, 0.10265578554597735, -0

In [63]:
# list_sentence_pairs = ['named','noun_phrase']
# dict_t_test ={}
# for model_type, loaded_model in dict_loaded_model.items():
#   dict_t_test[str(model_type)+"_noun_phrase"] = two_sample_test(dict_sentence_pairs =dict_noun_phrase_sentence_pair,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')
#   dict_t_test[str(model_type)+"_named"] = two_sample_test(dict_sentence_pairs =dict_list_named_sentence_pairs,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')


In [64]:
# dict_t_test.items()

In [65]:
# dict_t_test_noun_phrase_sentence_pair = two_sample_test(dict_sentence_pairs =dict_noun_phrase_sentence_pair,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')
# dict_t_test_named_sentence_pairs = two_sample_test(dict_sentence_pairs =dict_list_named_sentence_pairs,text_pipeline = text_pipeline, loaded_model = loaded_model, loaded_model_device = 'cpu')

In [66]:
# dict_result_named_sentence_pair ={}

# for key, value in dict_list_named_sentence_pairs.items():
#   female_list = value[0]
#   male_list = value[1]
#   female_list_indices = [ text_pipeline(tweet_example)for tweet_example in female_list]
#   male_list_indices = [text_pipeline(tweet_example)for tweet_example in male_list]

#   female_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in female_list]
#   male_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in male_list]
#   # for sentence in female_list:
#   #   female_list_output.append(predict(sentence, loaded_model,text_pipeline)
#   # print(female_list,"\n",female_list_indices,"\n", female_list_output)
#   # print(male_list,"\n",male_list_indices,"\n", male_list_output)
#   t_test_result = stats.ttest_rel(female_list_output, male_list_output)
#   dict_result_named_sentence_pair[key] = (t_test_result.statistic, t_test_result.pvalue,mean(female_list_output)-mean(male_list_output))
#   # print(type(stats.ttest_rel(female_list_output, male_list_output)))

#   # break

# print((dict_result_named_sentence_pair))

In [67]:
# #without named people
# dict_result_sentence_pair ={}
# # for key, value in dict_sentence_pair:
# #   if len(value[0])
# print(len(dict_sentence_pair))

# for key, value in dict_sentence_pair.items():
#   female_list = [value[0]]
#   male_list = [value[1]]
#   # if len(female_list)!=len(male_list):
#   #   print("key:", key)
#   #   print(female_list,"\n",male_list)
#   #   print(len(female_list),"-",len(male_list))
#   #   print(text_pipeline(female_list[0]),"\n",text_pipeline(male_list[0]))
#   #   break

#   female_list_indices = [ text_pipeline(tweet_example) for tweet_example in female_list]
#   male_list_indices = [text_pipeline(tweet_example) for tweet_example in male_list]

#   female_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in female_list]
#   male_list_output = [predict(sentence, loaded_model,text_pipeline,device= loaded_model_device) for sentence in male_list]
#   # for sentence in female_list:
#   #   female_list_output.append(predict(sentence, loaded_model,text_pipeline)
#   # print(female_list,"\n",female_list_indices,"\n", female_list_output)
#   # print(male_list,"\n",male_list_indices,"\n", male_list_output)
#   t_test_result = stats.ttest_rel(female_list_output, male_list_output)
#   dict_result_sentence_pair[key] = (t_test_result.statistic, t_test_result.pvalue,mean(female_list_output)-mean(male_list_output))
#   # print(type(stats.ttest_rel(female_list_output, male_list_output)))

#   # break

# print(dict_result_sentence_pair)

# Analysis of results (based on semval paper)

In [68]:
# dict_t_test_noun_phrase_sentence_pair
# dict_t_test_named_sentence_pairs

In [69]:
# len(dict_t_test_noun_phrase_sentence_pair),len(dict_t_test_named_sentence_pairs)

In [148]:
def analysis_t_test(dict_t_test_sentence_pairs, threshold = 0.05):
  list_output =[]
  for key, test_output in dict_t_test_sentence_pairs.items():
    significant=True
    t_statistic = test_output[0]
    p_value = test_output[1]
    f_m_diff = test_output[2]
    if (float(p_value) > float(threshold) or float(p_value) == float(threshold)):
      significant=False
      category = 'f_equals_m'
    else:
      significant=True
      
      if f_m_diff > 0:
        category='f_high_m_low'
      else:
        category = 'f_low_m_high' 
    list_output.append([key,t_statistic,p_value,significant,f_m_diff,category])
    
  df_columns = ['key','t_statistic','p_value', 'significant','delta','category']
  df_output = pd.DataFrame(list_output, columns = df_columns)

  list_category = list(df_output['category'].unique())
  list_statistics =[]
  for category in list_category:
    df_temp = df_output[df_output['category']==category]
    average = df_temp['delta'].mean()
    # print(category,len(df_temp), average)
    list_statistics.append([category,len(df_temp), average])
  df_statistics = pd.DataFrame(list_statistics, columns = ['category', 'num_pairs','average_difference'])
  return df_statistics


# print(analysis_t_test(dict_t_test_noun_phrase_sentence_pair))
# print(analysis_t_test(dict_t_test_named_sentence_pairs))


In [ ]:
{'EI_anger': {
    'non_dann': {
        'original_noun_phrase': {
            0: (0.1998956871564016, 0.8415904073105785, 3.9207107490946136e-05)}}, 
    'dann': {
        'original_noun_phrase': {
            0: (-3.055498578204014, 0.002288182511007486, -0.0005662351846695279)}}}, 'EI_sadness': {'non_dann': {'original_noun_phrase': {0: (0.697356464894513, 0.4856923878774828, 0.0001477275302426695)}}, 'dann': {'original_noun_phrase': {0: (2.6524225900747918, 0.008079505029578445, 0.0012747823571165329)}}}, 'EI_fear': {'non_dann': {'original_noun_phrase': {0: (-1.897698720306802, 0.057935765936073504, -0.0004837316667868352)}}, 'dann': {'original_noun_phrase': {0: (-4.093950892599476, 4.476596480751689e-05, -0.0010545446744395504)}}}, 'EI_joy': {'non_dann': {'original_noun_phrase': {0: (-1.2515370594455935, 0.210942010423521, -0.0002394391637708937)}}, 'dann': {'original_noun_phrase': {0: (-6.649931016376002, 4.1525086430524637e-11, -0.0012206101997030983)}}}, 'V': {'non_dann': {'original_noun_phrase': {0: (-0.6159244491387837, 0.5380417924786083, -0.00019533265795973476)}}, 'dann': {'original_noun_phrase': {0: (-1.5534286604033418, 0.12054065128353071, -0.0005289117702179658)}}}}


In [149]:
dict_statistics={}
for name, dict_model_type_sentence_pair in dict_t_test.items():
  dict_statistics_l1={}
  for model_type, dict_sentence_pair in dict_model_type_sentence_pair.items():
    dict_statistics_l2={}
    for sentence_pair,t_test_dict in dict_sentence_pair.items():
      df_statistics = analysis_t_test(t_test_dict, threshold = 0.05)
      print(name+"_"+model_type+"_"+sentence_pair)
      print(df_statistics)
      print(50*"=")
      dict_statistics_l2[sentence_pair] = df_statistics
    dict_statistics_l1[model_type] = dict_statistics_l2
  dict_statistics[name] = dict_statistics_l1
  # df_statistics = analysis_t_test(t_test_dict, threshold = 0.05)
  # dict_statistics[model_type_sentence_pair_name] = df_statistics

EI_anger_non_dann_named
       category  num_pairs  average_difference
0    f_equals_m        137           -0.001165
1  f_low_m_high          7           -0.001309
EI_anger_non_dann_noun_phrase
     category  num_pairs  average_difference
0  f_equals_m        144            0.000039
EI_anger_non_dann_original_noun_phrase
     category  num_pairs  average_difference
0  f_equals_m          1            0.000039
EI_anger_dann_named
       category  num_pairs  average_difference
0    f_equals_m        133           -0.001450
1  f_low_m_high         11           -0.001382
EI_anger_dann_noun_phrase
     category  num_pairs  average_difference
0  f_equals_m        144           -0.000566
EI_anger_dann_original_noun_phrase
       category  num_pairs  average_difference
0  f_low_m_high          1           -0.000566
EI_sadness_non_dann_named
     category  num_pairs  average_difference
0  f_equals_m        144           -0.002018
EI_sadness_non_dann_noun_phrase
     category  num_pairs  averag

In [146]:
print(dict_statistics)

{'EI_anger': {'non_dann': {'original_noun_phrase':      category  num_pairs  average_difference
0  f_equals_m          1            0.000039}, 'dann': {'original_noun_phrase':        category  num_pairs  average_difference
0  f_low_m_high          1           -0.000566}}, 'EI_sadness': {'non_dann': {'original_noun_phrase':      category  num_pairs  average_difference
0  f_equals_m          1            0.000148}, 'dann': {'original_noun_phrase':        category  num_pairs  average_difference
0  f_high_m_low          1            0.001275}}, 'EI_fear': {'non_dann': {'original_noun_phrase':      category  num_pairs  average_difference
0  f_equals_m          1           -0.000484}, 'dann': {'original_noun_phrase':        category  num_pairs  average_difference
0  f_low_m_high          1           -0.001055}}, 'EI_joy': {'non_dann': {'original_noun_phrase':      category  num_pairs  average_difference
0  f_equals_m          1           -0.000239}, 'dann': {'original_noun_phrase':        ca

In [71]:
# dict_statistics={}
# for model_type_sentence_pair_name, t_test_dict in dict_t_test.items():
#   df_statistics = analysis_t_test(t_test_dict, threshold = 0.05)
#   dict_statistics[model_type_sentence_pair_name] = df_statistics


In [72]:
for model_type_sentence_pair_name, df_statistics in dict_statistics.items():
  print(model_type_sentence_pair_name,"\n",df_statistics)
  print(50*"=")

non_dann_noun_phrase 
        category  num_pairs  average_difference
0    f_equals_m        126            0.005233
1  f_high_m_low         18            0.026457
non_dann_named 
        category  num_pairs  average_difference
0    f_equals_m        124            0.008112
1  f_high_m_low         20            0.023590
dann_noun_phrase 
        category  num_pairs  average_difference
0    f_equals_m        124            0.006045
1  f_high_m_low         20            0.021322
dann_named 
        category  num_pairs  average_difference
0    f_equals_m        121            0.008845
1  f_high_m_low         23            0.020125
